In [1]:
!pip install transformers
!pip install evaluate
!pip install torch
!pip install sentencepiece
!pip install protobuf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.w

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Read extracted text from file

In [2]:
file = open("drive/MyDrive/Colab Notebooks/extracted_text.txt", "r", encoding="utf-8")
long_text = file.read()
print(long_text)
file.close()

Y. FEI ET AL.: LEARNING WITH EXPERTS FOR FINE-GRAINED CATEGORY DISCOVERY 1
XCon: Learning with Experts for
Fine-grained Category Discovery
Yixin Fei1
yixin.feiyx@gmail.com
Zhongkai Zhao1
zhongkai.zhaok@gmail.com
Siwei Yang1,3
swyang.ac@gmail.com
Bingchen Zhao2,3
zhaobc.gm@gmail.com
1 Tongji University
Shanghai, China
2 University of Edinburgh,
Edinburgh, UK
3 LunarAI
Abstract
We address the problem of generalized category discovery (GCD) in this paper, i.e.
clustering the unlabeled images leveraging the information from a set of seen classes,
where the unlabeled images could contain both seen classes and unseen classes. The seen
classes can be seen as an implicit criterion of classes, which makes this setting different
from unsupervised clustering where the cluster criteria may be ambiguous. We mainly
concern the problem of discovering categories within a ﬁne-grained dataset since it is one
of the most direct applications of category discovery, i.e. helping experts discover novel
conce

# Split text into trunks

In [3]:
import math
import torch
from transformers import BartTokenizer

# 设置设备为 GPU
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def chunk_text_into_tokens(text, tokenizer, max_tokens=500):
    tokens = tokenizer.tokenize(text)
    token_chunks = [tokens[i:i + max_tokens] for i in range(0, len(tokens), max_tokens)]
    count = len(token_chunks)
    return token_chunks, count

max_text_length = 4096

# Model: BART

In [4]:
def summarize_chunks(token_chunks, max_sum_length, min_sum_length, tokenizer, model):
    summaries = []

    for chunk in token_chunks:
        chunk_text = tokenizer.convert_tokens_to_string(chunk)
        inputs = tokenizer(chunk_text, return_tensors='pt', padding=True, truncation=True, max_length=1024)
        # inputs = {key: tensor.to(device) for key, tensor in inputs.items()}  # Move inputs to device
        summary_ids = model.generate(**inputs, max_length=max_sum_length, min_length=min_sum_length, num_beams=4, early_stopping=True)
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        summaries.append(summary)

    return summaries

def concatenate_summaries(summaries):
    return " ".join(summaries)

In [ ]:
%%time

from transformers import BartTokenizer, BartForConditionalGeneration

tokenizer_bart = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
model_bart = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")

text_chunks, num_chunk = chunk_text_into_tokens(long_text, tokenizer_bart)
max_sum_length = min(500, math.floor(max_text_length / num_chunk))
min_sum_length = math.floor(0.5 * max_sum_length)

chunk_summaries_bart = summarize_chunks(text_chunks, max_sum_length, min_sum_length, tokenizer_bart, model_bart)

final_summary_bart = concatenate_summaries(chunk_summaries_bart)

tokens_bart = tokenizer_bart.tokenize(final_summary_bart)
number_of_tokens_bart = len(tokens_bart)
print("number_of_tokens: ", number_of_tokens_bart)

print(final_summary_bart)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

number_of_tokens:  2337
Y. FEI ET AL.: LEARNING WITH EXPERTS FOR FINE-GRAINED CATEGORY DISCOVERY 1                XCon: Learning with Experts forFine-grained Category Discovery. Yixin Fei: We address the problem of generalized category discovery (GCD) in this paper, i.e. leveraging the information from a set of seen classes. We present a novel method called Expert-Contrastive Learning. -                sion problems where large-scale human annotations are available, such as image recogni-                tion [5] and object detection [23]. However, collecting a dataset at scales like ImageNet or COCO is not always possible. Consider the scenario of ﬁne-grained recognition such as birdspecies recognition or medical image analysis, where the annotations require expert knowl-                edge which could be costly to collect. In ﬁne-grained category discovery, the main challenge is the large inter-class similarity and intra-classvariance. Different classes may require the model to learn

In [ ]:
with open("drive/MyDrive/Colab Notebooks/summary_bart.txt", "w", encoding="utf-8") as file:
    file.write(final_summary_bart)

# Model: T5

In [ ]:
def summarize_chunks_T5(chunks, max_sum_length, min_sum_length, tokenizer, model):
    summaries = []

    for chunk in chunks:
        chunk_text = tokenizer.convert_tokens_to_string(chunk)
        inputs = tokenizer("summarize: "+chunk_text, return_tensors='pt', padding=True, truncation=True, max_length=1024)
        summary_ids = model.generate(**inputs, max_length=max_sum_length, min_length=min_sum_length, num_beams=4, early_stopping=True)
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        summaries.append(summary)

    return summaries

In [ ]:
%%time

from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer_T5= T5Tokenizer.from_pretrained("t5-base", legacy=False)
model_T5 = T5ForConditionalGeneration.from_pretrained("t5-base")

text_chunks, num_chunk = chunk_text_into_tokens(long_text, tokenizer_T5)
max_sum_length = min(500, math.floor(max_text_length / num_chunk))
min_sum_length = math.floor(0.5 * max_sum_length)

chunk_summaries_T5 = summarize_chunks_T5(text_chunks, max_sum_length, min_sum_length, tokenizer_T5, model_T5)

final_summary_T5 = concatenate_summaries(chunk_summaries_T5)

tokens_T5 = tokenizer_T5.tokenize(final_summary_T5)
number_of_tokens_T5 = len(tokens_T5)
print("number_of_tokens: ", number_of_tokens_T5)

print(final_summary_T5)

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

number_of_tokens:  2075
we present a novel method called Expert-Contrastive Learning (XCon) to help the model to learn fine-grained discriminative features from the images. deep learning models have achieved super-human performance on many computer vi- sion problems. our method is based on the results of deep learning experiments on fine-grained datasets. it is hoped that deep learning will be able to improve the accuracy of deep learning models. the problem of generalized category discovery was recently formalized in [26]. aim is to discover categories within the unlabeled data by leveraging the information  2022. the copyright of this document resides with its authors. it may be distributed unchanged freely in print or electronic forms. cs.v:2208.01898v1 [cs.CV] 3 Aug 2022 2 Y. FEI ET AL. field experts are interested in the fine-grained concepts in real applications. we propose a method that can learn discriminative features for fine-grained category discovery. we partition the data 

In [ ]:
with open("drive/MyDrive/Colab Notebooks/summary_T5.txt", "w", encoding="utf-8") as file:
    file.write(final_summary_T5)

# Model: LED

In [ ]:
%%time
from transformers import LEDTokenizer, LEDForConditionalGeneration

tokenizer_LED = LEDTokenizer.from_pretrained("allenai/led-base-16384")
model_LED = LEDForConditionalGeneration.from_pretrained("allenai/led-base-16384")

text_chunks, num_chunk = chunk_text_into_tokens(long_text, tokenizer_LED)
max_sum_length = min(500, math.floor(max_text_length / num_chunk))
min_sum_length = math.floor(0.5 * max_sum_length)

chunk_summaries_LED = summarize_chunks(text_chunks, max_sum_length, min_sum_length, tokenizer_LED, model_LED)

final_summary_LED = concatenate_summaries(chunk_summaries_LED)

tokens_LED = tokenizer_LED.tokenize(final_summary_LED)
number_of_tokens_LED = len(tokens_LED)
print("number_of_tokens: ", number_of_tokens_LED)

print(final_summary_LED)

tokenizer_config.json:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/648M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

Input ids are automatically padded from 502 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 379 to 1024 to be a multiple of `config.attention_window`: 1024


number_of_tokens:  4033
Y. FEI ET AL.: LEARNING WITH EXPERTS FOR FINE-GRAINED CATEGORY DISCOVERY 1 UNCLASSIFIEDXCon: Learning with Experts for ﬁne-Fine-grained Category Discovery 1 UNCLASSIFIEDYixin Fei1,3 UNCLASSIFIEDyixin.feiyx@gmail.com UNCLASSIFIEDZhongkai Zhao1,3 UNCLASSIFIEDZzhongkai.zhaok@gmail.com UNCLASSIFIEDSiwei Yang1,3 UNCLASSIFIEDswyang.ac@gmail.com UNCLASSIFIEDBingchen Zhao2,3 UNCLASSIFIEDzhaobc.gm@gmail.com UNCLASSIFIED1 Tongji University,Shanghai, China UNCLASSIFIED2 University of Edinburgh,Shanghai, UK UNCLASSIFIED3 LunarAI UNCLASSIFIED1 LunarAI UNCLASSIFIEDAbstract UNCLASSIFIEDWe address the problem of generalized category discovery (GCD) in this paper - encountsion problems where large-scale human annotations are available, such as image recogni-tion [5] and object detection [23]. However, collecting a dataset at scales like ImageNet or UNCLASSIFIEDCOCO is not always possible. Consider the scenario of ﬁne-grained recognition such as bird-species recognition or medica

In [ ]:
with open("drive/MyDrive/Colab Notebooks/summary_LED.txt", "w", encoding="utf-8") as file:
    file.write(final_summary_LED)

# Model: Pegasus

In [ ]:
%%time

from transformers import PegasusForConditionalGeneration, AutoTokenizer

tokenizer_Pegasus = AutoTokenizer.from_pretrained("google/pegasus-xsum")
model_Pegasus = PegasusForConditionalGeneration.from_pretrained("google/pegasus-xsum")

text_chunks, num_chunk = chunk_text_into_tokens(long_text, tokenizer_Pegasus)

max_sum_length = min(500, math.floor(max_text_length / num_chunk))
min_sum_length = math.floor(0.5 * max_sum_length)

chunk_summaries_Pegasus = summarize_chunks(text_chunks, max_sum_length, min_sum_length, tokenizer_Pegasus, model_Pegasus)

final_summary_Pegasus = concatenate_summaries(chunk_summaries_Pegasus)

tokens_Pegasus = tokenizer_Pegasus.tokenize(final_summary_Pegasus)
number_of_tokens_Pegasus = len(tokens_Pegasus)
print("number_of_tokens: ", number_of_tokens_Pegasus)

print(final_summary_Pegasus)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/87.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.52M [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/259 [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (8763 > 512). Running this sequence through the model will result in indexing errors


number_of_tokens:  3006
This paper presents a novel method for learning fine-grained category discovery using expert knowledge from a set of seen classes, where the unlabeled images could contain both seen classes and unseen classes, and Experiments on fine- grained datasets show a clear improved performance over the previous best methods, indicating the effectiveness of our method.<n> Experiments on fine- grained datasets show a clear improved performance over the previous best methods, indicating the effectiveness of our method., Zhixin Fei, Zhongkai Zhao, Siwei Yang, and Bingchen Zhao, "Expert-Contrastive Learning (XCon)" The aim of this paper is to develop a method for learning fine-grained generalized category discovery by clustering the un-supervised data set by the criteria implicitly defined in the data set, and to show how this method can be used by experts for the discovery of new classes in the un-supervised data set, and to show how this method can be used by experts for th

In [ ]:
with open("drive/MyDrive/Colab Notebooks/summary_Pegasus.txt", "w", encoding="utf-8") as file:
    file.write(final_summary_Pegasus)

# Model: BigBirdPegasus

In [ ]:
%%time

from transformers import AutoTokenizer, BigBirdPegasusForConditionalGeneration

tokenizer_BigBird = AutoTokenizer.from_pretrained("google/bigbird-pegasus-large-arxiv")
model_BigBird = BigBirdPegasusForConditionalGeneration.from_pretrained("google/bigbird-pegasus-large-arxiv")

text_chunks, num_chunk = chunk_text_into_tokens(long_text, tokenizer_BigBird)
max_sum_length = min(500, math.floor(max_text_length / num_chunk))
min_sum_length = math.floor(0.5 * max_sum_length)

chunk_summaries_BigBird = summarize_chunks(text_chunks, max_sum_length, min_sum_length, tokenizer_BigBird, model_BigBird)

final_summary_BigBird = concatenate_summaries(chunk_summaries_BigBird)

tokens_BigBird = tokenizer_BigBird.tokenize(final_summary_BigBird)
number_of_tokens_BigBird = len(tokens_BigBird)
print("number_of_tokens: ", number_of_tokens_BigBird)

print(final_summary_BigBird)

Token indices sequence length is longer than the specified maximum sequence length for this model (8762 > 4096). Running this sequence through the model will result in indexing errors
Attention type 'block_sparse' is not possible if sequence_length: 501 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blocks * config.block_size + additional buffer: config.num_random_blocks * config.block_size = 704 with config.block_size = 64, config.num_random_blocks = 3. Changing attention type to 'original_full'...


number_of_tokens:  3367
this paper addresses the problem of finding the interclass similarity of an unlabeled set of images.<n> we show that it is possible to learn the interclass similarity of an unlabeled set of images using k - learning, where the performance of the method improves with the number of features on the learning set. <n> [ [ section ] ] in this paper we address the problem of finding the interclass similarity of an unlabeled set of images.<n> we show that it is possible to learn the interclass similarity of an unlabeled set of images using k - learning, where the performance of the method improves with the number of features on the learning set. <n> [ [ section ] ] in this paper we address the problem of finding the interclass similarity of an unlabeled set of images.<n> we show that it is possible to learn the interclass similarity of an unlabeled set of images using k - learning, where the performance of the method improves with the number of features on the learning 

In [ ]:
with open("drive/MyDrive/Colab Notebooks/summary_BigBird.txt", "w", encoding="utf-8") as file:
    file.write(final_summary_BigBird)

# Model: ProphetNet

In [5]:
%%time

from transformers import AutoTokenizer, ProphetNetForConditionalGeneration

tokenizer_prophetNet = AutoTokenizer.from_pretrained("microsoft/prophetnet-large-uncased")
model_prophetNet = ProphetNetForConditionalGeneration.from_pretrained("microsoft/prophetnet-large-uncased")

text_chunks, num_chunk = chunk_text_into_tokens(long_text, tokenizer_prophetNet)
max_sum_length = min(500, math.floor(max_text_length / num_chunk))
min_sum_length = math.floor(0.5 * max_sum_length)

chunk_summaries_prophetNet = summarize_chunks(text_chunks, max_sum_length, min_sum_length, tokenizer_prophetNet, model_prophetNet)

final_summary_prophetNet = concatenate_summaries(chunk_summaries_prophetNet)

tokens_prophetNet = tokenizer_prophetNet.tokenize(final_summary_prophetNet)
number_of_tokens_prophetNet = len(tokens_prophetNet)
print("number_of_tokens: ", number_of_tokens_prophetNet)

print(final_summary_prophetNet)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/141 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:365: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


prophetnet.tokenizer:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.57G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

number_of_tokens:  3081
in this paper, we address the problem of generalized category discovery ( gcd ). 1. 1 xcon : introduction to category discovery we present a novel method called expert - contrastive learning to help the model to learn discriminative features from the images. 2. 1 in this case, we present the method of category discovery to the models to the model, where the model can be seen to be an example of a case where the models can be used to mine useful information from the datasets. 3. 1 it is difﬁcult for the model to assume that the collected annotations are accurate and the annotation is accurate, but also it is costly to collect because of the time and expense involved, and it is not possible to collect enough annotates to be accurate, and also it could be expensive to collect. this document resides with its authors, where the author is the author of this document, and the authors are the authors. the authors reside with their authors, and this document is distribut

In [6]:
with open("drive/MyDrive/Colab Notebooks/summary_prophetNet_CPU.txt", "w", encoding="utf-8") as file:
    file.write(final_summary_prophetNet)

# Model: LlaMa2

# Model: Mistral

# Model: Gemma

# Read GPT4 generated summary as standard for comparison

In [4]:
file = open("drive/MyDrive/Colab Notebooks/gpt_summary.txt", "r", encoding="utf-8")
gpt_summary = file.read()
print(gpt_summary)
file.close()
references = gpt_summary

The paper "XCon: Learning with Experts for Fine-grained Category Discovery" by Yixin Fei et al. introduces a novel approach to address the problem of Generalized Category Discovery (GCD). GCD aims to cluster unlabeled images by leveraging information from a set of seen (labeled) classes, a task especially challenging within fine-grained datasets where classes are highly similar to one another. Traditional methods struggle with large inter-class similarities and intra-class variances, often leading to suboptimal performance due to irrelevant cues being used for classification. The proposed solution, Expert-Contrastive Learning (XCon), enhances the model's ability to discern fine-grained discriminative features by initially partitioning the dataset into sub-datasets using k-means clustering and subsequently applying contrastive learning within these partitions.

Introduction
With the advent of deep learning, there's been significant progress in computer vision tasks where large-scale ann

# Criteria: ROUGE score

In [5]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=d866842d88e93df36d209d167c3d84343453909f68a5d30b058ae857b9753f91
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [8]:
import os
import evaluate

rouge = evaluate.load("rouge")

folder_path = "drive/MyDrive/Colab Notebooks/result"
if not os.path.exists(folder_path):
    print("Folder does not exist:", folder_path)
else:
    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):
            file_path = os.path.join(folder_path, filename)
            file = open(file_path, "r", encoding="utf-8")
            predictions = file.read()
            file.close()
            rouge_scores = rouge.compute(predictions=[predictions], references=[references])
            print("File:", filename, "ROUGE Score:", rouge_scores)

File: summary_bart.txt ROUGE Score: {'rouge1': 0.4049513704686119, 'rouge2': 0.15309734513274337, 'rougeL': 0.16357206012378428, 'rougeLsum': 0.2961980548187445}
File: summary_T5.txt ROUGE Score: {'rouge1': 0.41365853658536583, 'rouge2': 0.1357421875, 'rougeL': 0.14341463414634145, 'rougeLsum': 0.2897560975609756}
File: summary_LED.txt ROUGE Score: {'rouge1': 0.32084805653710247, 'rouge2': 0.10396039603960396, 'rougeL': 0.127208480565371, 'rougeLsum': 0.22756183745583042}
File: summary_Pegasus.txt ROUGE Score: {'rouge1': 0.27896291434197573, 'rouge2': 0.09326765188834155, 'rougeL': 0.11880538234328848, 'rougeLsum': 0.19494584837545129}
File: summary_BigBird.txt ROUGE Score: {'rouge1': 0.21223765888264856, 'rouge2': 0.05087252292221235, 'rougeL': 0.0975465563109666, 'rougeLsum': 0.15075376884422112}
File: summary_prophetNet.txt ROUGE Score: {'rouge1': 0.294018249408584, 'rouge2': 0.08319242475481908, 'rougeL': 0.11220006759040219, 'rougeLsum': 0.20141939844542078}


1. ROUGE-1（unigram）：计算自动生成摘要和参考摘要中的单个词的重叠率。ROUGE-1 以单个词为单位进行匹配，衡量了单个词的重叠情况。
2. ROUGE-2（bigram）：计算自动生成摘要和参考摘要中的二元词组（即相邻的两个词）的重叠率。ROUGE-2 考虑了相邻词之间的匹配情况。
3. ROUGE-L：计算自动生成摘要和参考摘要之间最长公共子序列的长度，以此衡量两者之间的相似度。ROUGE-L 更加注重序列的匹配情况，而不仅仅是单个词或词组的匹配。
4. ROUGE-W：ROUGE-W 考虑了匹配词组的权重，即考虑了词组的长度。这个变体通常用于评估摘要生成任务中的长文本。

# Criteria: BLEU score

In [13]:
bleu = evaluate.load("bleu")
for filename in os.listdir(folder_path):
  if filename.endswith(".txt"):
    file_path = os.path.join(folder_path, filename)
    file = open(file_path, "r", encoding="utf-8")
    predictions = file.read()
    file.close()

    bleu_score = bleu.compute(predictions=[predictions], references=[references])
    print("File:", filename, "BLEU Score:", bleu_score)

File: summary_bart.txt BLEU Score: {'bleu': 0.0595763060619629, 'precisions': [0.27472527472527475, 0.09375, 0.03126809496236248, 0.015643105446118192], 'brevity_penalty': 1.0, 'length_ratio': 2.3208053691275166, 'translation_length': 1729, 'reference_length': 745}
File: summary_T5.txt BLEU Score: {'bleu': 0.048067770875327424, 'precisions': [0.3006802721088435, 0.08032675289312458, 0.02316076294277929, 0.00954328561690525], 'brevity_penalty': 1.0, 'length_ratio': 1.9731543624161074, 'translation_length': 1470, 'reference_length': 745}
File: summary_LED.txt BLEU Score: {'bleu': 0.035149683575393935, 'precisions': [0.20784490532010821, 0.06089309878213803, 0.016696750902527077, 0.007223476297968397], 'brevity_penalty': 1.0, 'length_ratio': 2.9771812080536915, 'translation_length': 2218, 'reference_length': 745}
File: summary_Pegasus.txt BLEU Score: {'bleu': 0.02900809358033149, 'precisions': [0.18307006885378696, 0.04943273905996758, 0.013781921361978111, 0.0056772100567721], 'brevity_p

1. bleu:表示自动生成的文本与参考文本之间的相似度。
2. precisions:计算 BLEU 分数时得到的各个 n-gram 的精确度（precision）值，其中包括了1-gram、2-gram、3-gram 和 4-gram 的精确度值。
3. brevity_penalty:计算 BLEU 分数时得到的长度惩罚（brevity penalty）值，用于惩罚生成的摘要比参考摘要短的情况。
4. length_ratio:自动生成的文本长度与参考文本长度之比，表示生成的摘要相对于参考摘要的长度。通常情况下，长度比值越大，表示生成的摘要越长。
5. translation_length:自动生成的文本的长度（单词数或字符数）。
6. reference_length:参考文本的长度（单词数或字符数）。

# Criteria: METEOR score

In [15]:
meteor = evaluate.load("meteor")
for filename in os.listdir(folder_path):
  if filename.endswith(".txt"):
    file_path = os.path.join(folder_path, filename)
    file = open(file_path, "r", encoding="utf-8")
    predictions = file.read()
    file.close()

    meteor_score = meteor.compute(predictions=[predictions], references=[references])
    print("File:", filename, "METEOR Score:", meteor_score)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


File: summary_bart.txt METEOR Score: {'meteor': 0.35274497019620843}
File: summary_T5.txt METEOR Score: {'meteor': 0.3381984335781387}
File: summary_LED.txt METEOR Score: {'meteor': 0.3244926932466287}
File: summary_Pegasus.txt METEOR Score: {'meteor': 0.29244985017537295}
File: summary_BigBird.txt METEOR Score: {'meteor': 0.23270890725564314}
File: summary_prophetNet.txt METEOR Score: {'meteor': 0.28323097310465906}
